In [1]:
!pip install -q sentence-transformers datasets pinecone-client faiss-cpu faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 25.1 MB/s eta 0:00:00


In [4]:
import datasets
squad = datasets.load_dataset('squad_v2', split='train')
squad

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [3]:
squad[0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [5]:
from sentence_transformers import InputExample
from tqdm import tqdm
train_samples = []
for row in squad:
  sample = InputExample(texts=[row['question'], row['context']])
  train_samples.append(sample)
len(train_samples)

130319

In [6]:
from sentence_transformers.datasets import NoDuplicatesDataLoader
batch_size = 2
loader = NoDuplicatesDataLoader(train_samples, batch_size=batch_size)
len(loader)

65159

In [9]:
from sentence_transformers import models, SentenceTransformer
mpnet = models.Transformer('microsoft/mpnet-base')
pool = models.Pooling(mpnet.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[mpnet, pool])
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [12]:
from sentence_transformers.losses import MultipleNegativesRankingLoss
loss_func = MultipleNegativesRankingLoss(model)

In [ ]:
epochs = 1
warmup_steps = (len(loader)*epochs*0.1)
model.fit(
    train_objectives = [(loader, loss_func)],
    epochs = epochs,
    warmup_steps = warmup_steps,
    show_progress_bar = True
)


In [15]:
model.save('mpnet-mnr-squad2')

In [17]:
squad_val = datasets.load_dataset('squad_v2', split='validation')
squad_val

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [21]:
import pandas as pd

temp = []
for row in squad_val:
  temp.append({
      'id': row['id'],
      'question': row['question'],
      'context': row['context']
  })
squad_val_df = pd.DataFrame(temp)
squad_val_df.head(5)

,id,question,context
0,56ddde6b9a695914005b9628,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...
2,56ddde6b9a695914005b962a,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...
3,56ddde6b9a695914005b962b,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...
4,56ddde6b9a695914005b962c,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...


In [24]:
cntx_df = squad_val_df.drop_duplicates(subset='context', keep='first')
cntx_df = cntx_df.drop(columns=['question'])
cntx_df['id'] = cntx_df['id'] + 'con'
cntx_df.head(3)

,id,context
0,56ddde6b9a695914005b9628con,The Normans (Norman: Nourmands; French: Norman...
9,56dddf4066d3e219004dad5fcon,"The Norman dynasty had a major political, cult..."
17,56dde0379a695914005b9636con,"The English name ""Normans"" comes from the Fren..."


In [26]:
squad_val_df = squad_val_df.merge(cntx_df, on='context', how='inner')

In [27]:
squad_val_df.head(2)

,id_x,question,context,id_y
0,56ddde6b9a695914005b9628,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9628con
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9628con


In [31]:
ir_queries = {row['id_x']: row['question'] for _,row in squad_val_df.iterrows()}
ir_cntx = {row['id_y']: row['context'] for _,row in squad_val_df.iterrows()}

In [34]:
from collections import defaultdict
ir_relevant_docs = defaultdict(set)
for _,row in squad_val_df.iterrows():
  ir_relevant_docs[row['id_x']].add(row['id_y'])

In [46]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
ir_eval = InformationRetrievalEvaluator(
  queries=ir_queries,
  corpus=ir_cntx,
  relevant_docs=ir_relevant_docs,
  show_progress_bar=True
)

In [ ]:
ir_eval(model)

Batches:   0%|          | 0/372 [00:00<?, ?it/s]

In [ ]:
qa_existing_model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')

In [ ]:
ir_eval(model)